In [1]:
full_state = {'left.x': 0.4678605021319262,
'left.y': 0.2758423627645942,
'left.z': 1.4101639137209923,
'box.x': 0.6724999615580194,
'box.y': 0.12650004790603953,
'box.z': 0.7750041578049789}

In [6]:
left_diss = (full_state['left.x'] - full_state['box.x'])**2 + (full_state['left.y'] - full_state['box.y'])**2 + (full_state['left.z'] - full_state['box.z'])**2
left_diss

0.46760835089680497

In [10]:
full_state['a'] = 1.0

In [11]:
full_state

{'a': 1.0,
 'box.x': 0.6724999615580194,
 'box.y': 0.12650004790603953,
 'box.z': 0.7750041578049789,
 'left.x': 0.4678605021319262,
 'left.y': 0.2758423627645942,
 'left.z': 1.4101639137209923}

In [1]:
#!/usr/bin/env python
import os
import sys
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.join(currentdir, currentdir+'/../'))
sys.path.append(os.path.join(currentdir, currentdir+'/../RL_brain'))
sys.path.append(os.path.join(currentdir, currentdir+'/../RL_brain/pointnet2/'))
import tensorflow.contrib as tc
import tensorflow as tf
import numpy as np
import pointnet2.utils.tf_utils as tf_utils
from pointnet2.utils.pointnet_util import pointnet_sa_module

In [2]:
def get_model(point_cloud, is_training=True, bn_decay=None):
    """ PointNet 特征抽取网络, input is BxNx3, output.shape(batch_size, -1) """
    batch_size = point_cloud.shape[0]
    num_point = point_cloud.shape[1]
    end_points = {}
    l0_xyz = point_cloud
    l0_points = None
    end_points['l0_xyz'] = l0_xyz

    # Set abstraction layers
    # Note: When using NCHW for layer 2, we see increased GPU memory usage (in TF1.4).
    # So we only use NCHW for layer 1 until this issue can be resolved.
    l1_xyz, l1_points, l1_indices = pointnet_sa_module(l0_xyz, l0_points, npoint=512, radius=0.2, nsample=32, mlp=[64,64,128], mlp2=None, group_all=False, is_training=is_training, bn_decay=bn_decay, scope='layer1', use_nchw=True)
    l2_xyz, l2_points, l2_indices = pointnet_sa_module(l1_xyz, l1_points, npoint=128, radius=0.4, nsample=64, mlp=[128,128,256], mlp2=None, group_all=False, is_training=is_training, bn_decay=bn_decay, scope='layer2')
    l3_xyz, l3_points, l3_indices = pointnet_sa_module(l2_xyz, l2_points, npoint=None, radius=None, nsample=None, mlp=[256,512,1024], mlp2=None, group_all=True, is_training=is_training, bn_decay=bn_decay, scope='layer3')

    # Fully connected layers
    net = tf.reshape(l3_points, [batch_size, -1])
    return net, end_points

In [3]:
observe_pcl = tf.zeros((1,1024,3))
observe_image = tf.zeros((1,256,256,3))
trainable=True
is_training=True

In [4]:
normalizer_fn = tc.layers.layer_norm
conv_1 = tc.layers.conv2d(inputs=observe_image, num_outputs=32, kernel_size=3,
                                      stride=2, padding='valid', trainable=trainable, normalizer_fn=normalizer_fn)
conv_2 = tc.layers.conv2d(inputs=conv_1, num_outputs=32, kernel_size=3,
                                      stride=2, padding='valid', trainable=trainable, normalizer_fn=normalizer_fn)
conv_3 = tc.layers.conv2d(inputs=conv_2, num_outputs=32, kernel_size=3,
                                      stride=2, padding='valid', trainable=trainable, normalizer_fn=normalizer_fn)
conv_4 = tc.layers.conv2d(inputs=conv_3, num_outputs=56, kernel_size=5,
                                      stride=2, padding='valid', trainable=trainable, normalizer_fn=normalizer_fn)
conv_5 = tc.layers.conv2d(inputs=conv_4, num_outputs=128, kernel_size=5,
                                      stride=2, padding='valid', trainable=trainable, normalizer_fn=normalizer_fn)
conv_6 = tc.layers.conv2d(inputs=conv_5, num_outputs=256, kernel_size=5,
                                      stride=2, padding='valid', trainable=trainable, normalizer_fn=normalizer_fn)
flatten_layer_image = tf.layers.flatten(conv_6)

# pointnet2 抽取特征
flatten_layer_pcl, end_points = get_model(point_cloud=observe_pcl,
                                   is_training=is_training,
                                   bn_decay=None) # end_points 占时不知道干什么用

connected_layer = tf.concat([flatten_layer_image, flatten_layer_pcl], axis=1)

dense_1 = tf.layers.dense(connected_layer, units=512, activation=None, trainable=trainable)
dense_1 = tc.layers.layer_norm(dense_1, center=True, scale=True)
dense_1 = tf.nn.relu(dense_1)

dense_2 = tf.layers.dense(dense_1, units=256, activation=None, trainable=trainable)
dense_2 = tc.layers.layer_norm(dense_2, center=True, scale=True)
dense_2 = tf.nn.relu(dense_2)

dense_3 = tf.layers.dense(dense_2, units=256, activation=None, trainable=trainable)
dense_3 = tc.layers.layer_norm(dense_3, center=True, scale=True)
dense_3 = tf.nn.relu(dense_3)

dense_4 = tf.layers.dense(dense_3, units=256, activation=None, trainable=trainable)
dense_4 = tc.layers.layer_norm(dense_4, center=True, scale=True)
dense_4 = tf.nn.relu(dense_4)

action_output = tf.layers.dense(dense_4, units=6, activation=tf.nn.tanh,
                                kernel_initializer=tf.initializers.random_uniform(minval=-0.003,
                                                                                  maxval=0.003), trainable=trainable)

#输出(1,4)
action_output = action_output * np.array([0.05, 0.05, 0.05, np.radians(90),np.radians(90),np.radians(90)])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use keras.layers.dense instead.


In [5]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print sess.run(action_output)

[[0. 0. 0. 0. 0. 0.]]


In [1]:
import numpy as np
import random
p = np.zeros((1024,3))

In [11]:
import tensorflow as tf
a=tf.constant(np.tile(np.array([0,0,0]).reshape((1,1,3)), (32,2,5)),dtype=tf.float32) # (batch_size, 1, 3)

In [2]:
observe = {'pcl':np.array([[1,2],[3,4]]), 'image':np.array([[1,2],[3,4]])}

In [8]:
type(observe) == dict

True

In [10]:
def array_min2d(x): # 加入demo 时候也要注意 数据类型
    if type(x) == dict:
        return x
        pass
    else:
        x = np.array(x, dtype=np.float32)
        if x.ndim >= 2:
            return x
        else:
            return x.reshape(-1, 1)

In [33]:
elem_enumer = ((0, 'observe0_img'),(1, 'observe0_pcl'),
         (2, 'observe1_img'),(3, 'observe1_pcl'),
         (4, 'actions'),(5, 'rewards'),
         (6, 'full_states0'),(7, 'full_states1'),(8, 'terminals1'))

In [36]:
2**11

2048

In [31]:
for j, element_class in enumerate(a):
    print (j, element_class)

(0, 'observe0_img')
(1, 'observe0_pcl')
(2, 'observe1_img')
(3, 'observe1_pcl')
(4, 'actions')
(5, 'rewards')
(6, 'full_states0')
(7, 'full_states1')
(8, 'terminals1')


In [28]:
a = ['observe0_img','observe0_pcl','observe1_img','observe1_pcl','actions','rewards','full_states0','full_states1','terminals1']

In [1]:
#!/usr/bin/env python
from geo import *
import math

In [2]:
leap_2_base = FRAME(xyzrpy = [0.750000, 0.000000,
        -0.129000, 1.57079632679, 0, -1.57079632679])

In [7]:
leap_2_base.xyzrpy()

[0.75, 0.0, -0.129, 1.57079632679, -0.0, -1.57079632679]

In [9]:
leap_2_base.__mul__(FRAME(xyzrpy = [0.5, 0.000000,
    0.0, 0., 0., 0.])).xyzrpy()

[0.7500000000024483, -0.5, -0.129, 0.0, -0.0, -1.57079632679]

In [39]:
leap_2_base.__mul__(FRAME(xyzrpy = [0., 0.000000,
    -0.5, 0, 0, 0.])).xyzrpy()

[1.25,
 2.4482944300733737e-12,
 -0.1290000000024483,
 1.57079632679,
 -0.0,
 -1.57079632679]

In [42]:
FRAME(xyzrpy = [0., 0.000000,
    -0.5, 0, 0, 0.]).__mul__(base_2_leap).xyzrpy()

[0.75, 0.0, -0.629, 1.57079632679, -0.0, -1.57079632679]